<a href="https://colab.research.google.com/github/walysson21/Notebooks_Computer_Vision_Mestrado/blob/main/6_RGB_NDVI_AgricultureVisionU_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Carregamento do Dataset original

In [ ]:
import keras
import numpy as np
import cv2
from PIL import Image
import glob
import matplotlib.pyplot as plt
import tensorflow.keras

dataset_path = '/content/drive/MyDrive/ColabAux/Agriculture-Vision'
batch_size=32
img_size=256

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, train_im_path=None,train_nir_path=None,train_mask_path=None, train_mask_paths=None, augmentations=None, 
                 batch_size=batch_size,img_size=img_size, n_channels=4, shuffle=True, x_return=True, preprocessing=None):
        'Initialization'
        
        if train_mask_paths is None:
          self.train_mask_paths = glob.glob(train_mask_path+'/*')
        else:
          self.train_mask_paths = train_mask_paths

        self.preprocessing = preprocessing
        self.batch_size = batch_size
        self.train_im_path = train_im_path
        self.train_nir_path = train_nir_path
        self.train_mask_path = train_mask_path
        self.img_size = img_size
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.augment = augmentations
        self.x_return = x_return
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(len(self.train_mask_paths) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:min((index+1)*self.batch_size,len(self.train_mask_paths))]

        # Find list of IDs
        list_IDs_im = [self.train_mask_paths[k] for k in indexes]

        # Generate data
        if self.x_return:
          X, y = self.data_generation(list_IDs_im)

          if self.augment is None:
              return X,y #np.array(y)/255
          else:            
              im,mask = [],[]   
              for x,y in zip(X,y):
                  augmented = self.augment(image=x, mask=y)
                  im.append(augmented['image'])
                  mask.append(augmented['mask'])
              return np.array(im),np.array(mask)/255
        else:
          y = self.data_generation(list_IDs_im)
          return y #np.array(y)/255

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.train_mask_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def data_generation(self, list_IDs_im):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        if self.x_return:
          X = np.empty((len(list_IDs_im),self.img_size,self.img_size, self.n_channels))
        y = np.empty((len(list_IDs_im),self.img_size,self.img_size, 1))

        # Generate data
        for i, mask_path in enumerate(list_IDs_im):
            mask = np.array(Image.open(mask_path))

            if self.x_return:
              im_path = mask_path.replace(self.train_mask_path, self.train_im_path)
              im_path = im_path.replace('png', 'jpg')
              im_rgb = np.array(Image.open(im_path))
              im_rgb = cv2.resize(im_rgb,(self.img_size,self.img_size))

              im_nir_path = im_path.replace(self.train_im_path, self.train_nir_path)
              im_nir = np.array(Image.open(im_nir_path))
              im_nir = cv2.resize(im_nir,(self.img_size,self.img_size))[..., np.newaxis]

              ndvi = (im_nir - im_rgb[...,0][..., np.newaxis])/(im_nir.astype(np.float32) + im_rgb[...,0][..., np.newaxis].astype(np.float32) + 1e-6)
              #ndwi = (im_rgb[...,2][..., np.newaxis] - im_nir)/(im_nir.astype(np.float32) + im_rgb[...,2][..., np.newaxis].astype(np.float32) + 1e-6)
              #im = np.dstack((im_rgb, im_nir, ndvi, ndwi))

              #im = np.dstack((im_rgb, im_nir))
              im = np.dstack((im_rgb, ndvi))

              if len(im.shape)==2:
                  im = np.repeat(im[...,None],3,2)

              # Resize sample
              X[i,] = im #cv2.resize(im,(self.img_size,self.img_size))

            # Store class
            y[i,] = cv2.resize(mask,(self.img_size,self.img_size))[..., np.newaxis]
            y[y>0] = 255

        y[y>0] = 255
        y = np.uint8(y)
        y = np.array(y)/255
        X = np.uint8(X)
        y = tensorflow.keras.utils.to_categorical(y, num_classes=2, dtype="float32")
        if self.preprocessing:
          X = self.preprocessing(X)

        if self.x_return:
          return X, y
        else:
          return y

# Preparando o ambiente

In [ ]:
%env SM_FRAMEWORK=tf.keras
!pip install -U segmentation-models

env: SM_FRAMEWORK=tf.keras
     |████████████████████████████████| 51kB 7.5MB/s 


# Preparação dos dados original

In [ ]:
import numpy as np
from PIL import Image
import glob
import random
import pandas as pd

# PEGA AS MASCARAS COM A PRESENÇA DAS CLASSES.
# DEPOIS ACRESCENTA 20% DE MASCARAS SEM A PRESENÇA DA CLASSE 
def build_mask_paths(mask_path, percent_without_class=0.2, read_csv=None):
  paths_with_class = list()
  all_paths = glob.glob(mask_path+'/*')

  print('len(all_paths) = ', len(all_paths))

  if read_csv is None:
    for file in all_paths:
      im = np.array(Image.open(file))
      if im.__contains__(255):
        paths_with_class.append(file)
  else:
    df_paths = pd.read_csv(read_csv)
    paths_with_class = df_paths.values.tolist()
    paths_with_class = [i[0] for i in paths_with_class]
  
  print('len(paths_with_class) = ', len(paths_with_class))
  paths_without_class = list(set(all_paths) - set(paths_with_class))
  # pega 20% len(paths_with_class)
  paths_without_class = random.sample(paths_without_class, int(len(paths_with_class)*percent_without_class))
  print('len(paths_without_class) = ', len(paths_without_class))
  paths = paths_without_class + paths_with_class
  print('len(paths) = ', len(paths))
  paths = random.sample(paths, len(paths))

  return paths, paths_with_class, paths_without_class


In [ ]:
import pandas as pd
import segmentation_models as sm
from keras.callbacks import ModelCheckpoint
import random

img_size= 256#512
percents=3.
batch_size=8

classes = ['cloud_shadow', 'double_plant', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']

dataset_path = '/content/drive/MyDrive/ColabAux/Agriculture-Vision'
class_name = classes[5]

train_im_path = dataset_path+'/train/images/rgb'
train_nir_path = dataset_path+'/train/images/nir' 
train_mask_path = dataset_path+'/train/labels/'+class_name
test_im_path = dataset_path+'/val/images/rgb' 
test_nir_path = dataset_path+'/val/images/nir' 
test_mask_path = dataset_path+'/val/labels/'+class_name
csv_paths_train = '/content/drive/MyDrive/ColabAux/list_paths/list_path_train_'+class_name+'.csv'
csv_paths_val = '/content/drive/MyDrive/ColabAux/list_paths/list_path_val_'+class_name+'.csv'

#train_paths,z,zz = build_mask_paths(train_mask_path, percents, csv_paths_train)
#train_generator = DataGenerator(train_im_path=train_im_path, train_mask_path=train_mask_path, train_mask_paths=train_paths,
#                              batch_size=batch_size, img_size=img_size)

Segmentation Models: using `tf.keras` framework.


In [ ]:
len_val = 500

x_rgb_val = np.load('/content/drive/MyDrive/ColabAux/var_numpy/x_test.npy')[0:len_val].astype(np.float32)
x_nir_val = np.load('/content/drive/MyDrive/ColabAux/var_numpy/x_nir_test.npy')[0:len_val].astype(np.float32)


ndvi = (x_nir_val - x_rgb_val[...,0][..., np.newaxis])/(x_nir_val + x_rgb_val[...,0][..., np.newaxis] + 1e-6)
#ndwi = (x_rgb_val[...,2][..., np.newaxis] - x_nir_val)/(x_nir_val + x_rgb_val[...,2][..., np.newaxis] + 1e-6)
#x_val = np.empty((len(x_rgb_val),img_size,img_size,6))

x_val = np.empty((len(x_rgb_val),img_size,img_size,4))

for i in range(0, len(x_rgb_val)):
  #x_val[i,] = np.dstack((x_rgb_val[i,], x_nir_val[i,], ndvi[i,], ndwi[i,]))
  x_val[i,] = np.dstack((x_rgb_val[i,], ndvi[i,]))

x_rgb_val, x_nir_val, ndvi, ndwi = [], [], [], []

y_val = np.load('/content/drive/MyDrive/ColabAux/var_numpy/y_test_'+class_name+'.npy')[0:len_val]
y_val = tensorflow.keras.utils.to_categorical(y_val, num_classes=2, dtype="float32")

# Tentativa com o Hyperopt

In [ ]:
import segmentation_models as sm
import tensorflow as tf
import tensorflow.keras
import glob
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd

from hyperopt import rand, fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.mongoexp import MongoTrials

bb = [#'vgg16', 'vgg19', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 
      #'seresnet18', 'seresnet34', 
      'seresnet50', 'seresnet101', 'seresnet152', 
      #'resnext50', 'resnext101', 
      'seresnext50', 'seresnext101', 'senet154', 
      #'densenet121', 'densenet169', 'densenet201', 'inceptionv3', 'inceptionresnetv2', 
      #'mobilenet', 'mobilenetv2', 
      'efficientnetb0', 'efficientnetb1', 'efficientnetb2', 
      'efficientnetb3', 'efficientnetb4', 'efficientnetb5', 'efficientnetb6', 'efficientnetb7']
batch256 = [6, 8, 10, 12]
#batch512 = [2, 3, 4]
#decoder_block_type = ['upsampling', 'transpose']
slices_dataset = [1, 2, 3, 4, 5]
loss = [sm.losses.jaccard_loss, sm.losses.bce_jaccard_loss, sm.losses.binary_focal_jaccard_loss, 
        sm.losses.categorical_focal_jaccard_loss, sm.losses.cce_jaccard_loss,
        sm.losses.dice_loss, sm.losses.bce_dice_loss, sm.losses.binary_focal_dice_loss,
        sm.losses.categorical_focal_dice_loss, sm.losses.cce_dice_loss #,
        #sm.losses.categorical_focal_loss, sm.losses.binary_crossentropy
        ]

Definição do espaço de hiperparâmetros

In [ ]:
space = {'bb': hp.choice('bb', bb),
         'batch': hp.choice('batch', batch256),
         #'decoder_block_type': hp.choice('decoder_block_type', decoder_block_type),
         'slices_dataset': hp.choice('slices_dataset', slices_dataset),
         'loss': hp.choice('loss', loss),
         'epochs': 15,
         'img_size': 256
        }

Definição da função objetivo

In [ ]:
def objective(params):
  global df

  model = sm.Unet(params['bb'], encoder_weights='imagenet', classes=2, activation='softmax')
  model.compile(
      'Adam',
      loss=params['loss'],
      metrics=[sm.metrics.iou_score],
  )

  train_paths,p_with,p_without = build_mask_paths(train_mask_path, params['slices_dataset'] , csv_paths_train)
  train_paths = random.sample(p_with, int(len(p_with)*0.1)) + random.sample(p_without, int(len(p_without)*0.1))
  print('len(paths) * 0.1 = ', len(train_paths))
  train_generator = DataGenerator(train_im_path=train_im_path, train_mask_path=train_mask_path, train_mask_paths=train_paths,
                              batch_size=params['batch'], img_size=params['img_size'])

  validations = 6
  test_iou = [None]*validations
  for i in range(0, validations):
    history = model.fit_generator(train_generator,
                                  #validation_data=(x_val, y_val),
                                  epochs=params['epochs'],
                                  verbose=0
                                  )
    eval = model.evaluate(x_val, y_val)
    test_iou[i] = eval[1]
  #val_iou = max(history.history['val_iou_score'])
  #epoch_best_iou = history.history['val_iou_score'].index(val_iou) + 1

  val_iou = max(test_iou)
  epoch_best_iou = (test_iou.index(val_iou) + 1) * params['epochs']

  row = {'backbone':params['bb'], 'batch_size':params['batch'], 'func_loss':params['loss'].name, #'decoder_block_type':params['decoder_block_type'], 
         'val_iou':str(val_iou), 'epoch_best_iou':epoch_best_iou, 'slices_dataset':params['slices_dataset']}

  df = df.append(row, ignore_index=True)
  df = df.sort_values('val_iou', ascending=False)
  df.to_csv(r'/content/drive/MyDrive/ColabAux/hyperopt_unet256_'+class_name+'_2.csv')
  return (1 - val_iou)

Teste dos hiperparâmetros com o Hyperopt

In [ ]:
class_name

'waterway'

In [ ]:
max_evals = 22 #2 * len(bb) * len(batch)
df = pd.DataFrame()
trials = Trials()

best = fmin(objective, space, algo=rand.suggest, trials=trials, max_evals=max_evals)
loss = trials.best_trial['result']['loss']

df = df.sort_values('val_iou', ascending=False)
df.to_csv(r'/content/drive/MyDrive/ColabAux/hyperopt_unet256_'+class_name+'_2.csv')
df

len(all_paths) = 
12901
len(paths_with_class) = 
1769
len(paths_without_class) = 
8845
len(paths) = 
10614
len(paths) * 0.1 = 
1060
  0%|          | 0/22 [00:08<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '



A saída de streaming foi truncada nas últimas 5000 linhas.
 - ETA: 14s - loss: 0.4951 - iou_score: 0.5407

 17/139 [==>...........................]
 - ETA: 14s - loss: 0.4939 - iou_score: 0.5405

 18/139 [==>...........................]
 - ETA: 14s - loss: 0.4929 - iou_score: 0.5408

 19/139 [===>..........................]
 - ETA: 14s - loss: 0.5046 - iou_score: 0.5376

 20/139 [===>..........................]
 - ETA: 14s - loss: 0.4990 - iou_score: 0.5395

 21/139 [===>..........................]
 - ETA: 14s - loss: 0.4984 - iou_score: 0.5381


In [ ]:
df = df.sort_values('val_iou', ascending=False)
#df = pd.read_csv(r'/content/drive/MyDrive/ColabAux/results_hyperopt_256_T3.csv')
#del df['Unnamed: 0']
df #foi

# Grid Search

In [ ]:
import pandas as pd
import segmentation_models as sm
from keras.callbacks import ModelCheckpoint
import random

img_size=256 #512
percents=2
batch_size=6

dataset_path = '/content/drive/MyDrive/ColabAux/Agriculture-Vision'
class_name = 'cloud_shadow'

train_im_path = dataset_path+'/train/images/rgb' 
train_mask_path = dataset_path+'/train/labels/'+class_name
test_im_path = dataset_path+'/val/images/rgb' 
test_mask_path = dataset_path+'/val/labels/'+class_name
csv_paths_train = '/content/drive/MyDrive/ColabAux/list_paths/list_path_train_'+class_name+'.csv'
csv_paths_val = '/content/drive/MyDrive/ColabAux/list_paths/list_path_val_'+class_name+'.csv'

train_paths,p_with,p_without = build_mask_paths(train_mask_path, percents, csv_paths_train)
train_paths = random.sample(p_with, int(len(p_with)*0.05)) + random.sample(p_without, int(len(p_without)*0.05))
print(len(train_paths))
train_generator = DataGenerator(train_im_path=train_im_path, train_mask_path=train_mask_path, train_mask_paths=train_paths,
                              batch_size=batch_size, img_size=img_size)

len(all_paths) =  12901
len(paths_with_class) =  931
len(paths_without_class) =  1862
len(paths) =  2793
139


In [ ]:
x_val = np.load('/content/drive/MyDrive/ColabAux/var_numpy/256/x_test.npy')
y_val = np.load('/content/drive/MyDrive/ColabAux/var_numpy/256/y_test_'+class_name+'.npy')
y_val = tensorflow.keras.utils.to_categorical(y_val, num_classes=2, dtype="float32")

In [ ]:
epochs=90
df = pd.DataFrame({'Dataset / mIoU': ['Train mIoU', 'Test mIoU']})

batches = [2, 4, 6, 8, 10, 12]

for b in batches:
  model = sm.Unet('efficientnetb7', encoder_weights='imagenet', classes=2, activation='softmax')
  model.compile('Adam', loss=sm.losses.bce_jaccard_loss, metrics=[sm.metrics.iou_score])

  train_generator = DataGenerator(train_im_path=train_im_path, train_mask_path=train_mask_path, train_mask_paths=train_paths,
                              batch_size=b, img_size=img_size)

  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(x_val, y_val), verbose=1)

  val_iou = max(history.history['val_iou_score'])
  ind = history.history['val_iou_score'].index(val_iou)

  df[b] = [history.history['iou_score'][ind], val_iou]
  df.to_csv(r'/content/drive/MyDrive/ColabAux/best_unet_batche_256_T2.csv')

df

258441216/258434480 [==============================] - 4s 0us/step


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/90
70/70 [==============================] - 199s 2s/step - loss: 1.2787 - iou_score: 0.3491 - val_loss: 1.3345 - val_iou_score: 0.4510
Epoch 2/90
70/70 [==============================] - 35s 508ms/step - loss: 0.9210 - iou_score: 0.4407 - val_loss: 0.6772 - val_iou_score: 0.5119
Epoch 3/90
70/70 [==============================] - 35s 508ms/step - loss: 0.9172 - iou_score: 0.4553 - val_loss: 0.6654 - val_iou_score: 0.4967
Epoch 4/90
70/70 [==============================] - 35s 508ms/step - loss: 0.8854 - iou_score: 0.4698 - val_loss: 1.5519 - val_iou_score: 0.4915
Epoch 5/90
70/70 [==============================] - 35s 509ms/step - loss: 0.8870 - iou_score: 0.4819 - val_loss: 0.6938 - val_iou_score: 0.5055
Epoch 6/90
70/70 [==============================] - 35s 508ms/step - loss: 0.8553 - iou_score: 0.4740 - val_loss: 0.6161 - val_iou_score: 0.5285
Epoch 7/90
70/70 [==============================] - 35s 508ms/step - loss: 0.9553 - iou_score: 0.4594 - val_loss: 0.7263 - val_iou_s

,Dataset / mIoU,2,4,6,8,10,12
0,Train mIoU,0.739854,0.845941,0.875530,0.957257,0.913900,0.977785
1,Test mIoU,0.604451,0.616457,0.637352,0.631198,0.634656,0.627614


# Treinamento final

In [ ]:
class_name

'weed_cluster'

In [ ]:
import pandas as pd
import segmentation_models as sm
from keras.callbacks import ModelCheckpoint
import random

bb = 'efficientnetb7'
batch = 8
slices_dataset = 0 #'Full'
class_weights=[0,2]
loss = sm.losses.JaccardLoss(class_weights=class_weights)
loss = loss + sm.losses.binary_focal_loss
epochs = 1
img_size = 256
iou_max = 0

model = sm.Unet(bb, input_shape=(None, None, 4), encoder_weights=None, classes=2, activation='softmax')
model.compile(
    keras.optimizers.Adam(1.e-4),
    loss=loss,
    metrics=[sm.metrics.IOUScore(threshold=0.5), 
             sm.metrics.IOUScore(threshold=0.5, class_weights=[0,2],name='iou_class'),
             sm.metrics.IOUScore(threshold=0.5, class_weights=[2,0],name='iou_BG')],
)
'''
model.load_weights('/content/drive/MyDrive/ColabAux/Pesos/AgriVisFinal/unet256_nrgb_'+class_name+'_b5.hdf5')
#eval = model.evaluate(x_val, y_val)
#iou_max = eval[2]
'''
train_paths,p_with,p_without = build_mask_paths(train_mask_path, slices_dataset, csv_paths_train)
train_paths = random.sample(train_paths, int(len(train_paths)*0.2))

train_generator = DataGenerator(train_im_path=train_im_path, train_nir_path=train_nir_path, train_mask_path=train_mask_path, train_mask_paths=train_paths, 
                                batch_size=batch, img_size=img_size)

validations = 100
for i in range(0, validations):
  history = model.fit_generator(train_generator,
                                epochs=epochs,
                                verbose=1,
                                validation_data=(x_val, y_val), 
                                #callbacks=[checkpoint, reduce_lr]
                                )
  eval = model.evaluate(x_val, y_val)
  iou = eval[2]
  if iou > iou_max:
    iou_max = iou
    #model.save('/content/drive/MyDrive/ColabAux/Pesos/AgriVisFinal/unet256_nrgbvw_'+class_name+'_b0.hdf5')
  print('Avaliação - Época ' + str((i+1)*epochs))
  print('iou_max = ' + str(iou_max))
  print('')
  

/usr/local/lib/python3.7/dist-packages/keras_applications/imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  str(input_shape[-1]) + ' input channels.')


len(all_paths) =  12901
len(paths_with_class) =  8890
len(paths_without_class) =  0
len(paths) =  8890


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


16/16 [==============================] - 5s 310ms/step - loss: 0.9964 - iou_score: 0.4223 - iou_class: 0.0921 - iou_BG: 0.7525
Avaliação - Época 1
iou_max = 0.09209783375263214

16/16 [==============================] - 5s 309ms/step - loss: 0.9732 - iou_score: 0.4624 - iou_class: 0.1524 - iou_BG: 0.7725
Avaliação - Época 2
iou_max = 0.15235944092273712

16/16 [==============================] - 5s 309ms/step - loss: 0.8996 - iou_score: 0.4976 - iou_class: 0.2455 - iou_BG: 0.7497
Avaliação - Época 3
iou_max = 0.24548883736133575

16/16 [==============================] - 5s 309ms/step - loss: 0.9017 - iou_score: 0.4994 - iou_class: 0.2351 - iou_BG: 0.7636
Avaliação - Época 4
iou_max = 0.24548883736133575

16/16 [==============================] - 5s 309ms/step - loss: 0.8794 - iou_score: 0.5000 - iou_class: 0.2675 - iou_BG: 0.7325
Avaliação - Época 5
iou_max = 0.2675156891345978

16/16 [==============================] - 5s 309ms/step - loss: 0.9142 - iou_score: 0.4870 - iou_class: 0.2028 -

In [ ]:
model.save('/content/drive/MyDrive/ColabAux/Pesos/AgriVisFinal/unet256_nrgbvw_'+class_name+'_b0_overfit_iouc64.hdf5')

In [ ]:
model.load_weights('/content/drive/MyDrive/ColabAux/Pesos/AgriVisFinal/unet256_nrgb_'+class_name+'_b5.hdf5')

In [ ]:
eval = model.evaluate(x_val, y_val)

139/139 [==============================] - 17s 124ms/step - loss: 0.0238 - iou_score: 0.6111 - iou_class: 0.2459 - iou_BG: 0.9762
